In [1]:
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import *
import os
import numpy as np
import inspect
from pathlib import Path
import zipfile
import sys

In [2]:
def saveraster(gdal_data, filename, raster):
    rows = gdal_data.RasterYSize
    cols = gdal_data.RasterXSize

    outDs = gdal.GetDriverByName("GTiff").Create(filename, cols, rows, int(1), GDT_Float32)
    outBand = outDs.GetRasterBand(1)

    # write the data
    outBand.WriteArray(raster, 0, 0)
    # flush data to disk, set the NoData value and calculate stats
    outBand.FlushCache()
    outBand.SetNoDataValue(-9999)

    # georeference the image and set the projection
    outDs.SetGeoTransform(gdal_data.GetGeoTransform())
    outDs.SetProjection(gdal_data.GetProjection())

In [3]:
def create_patches(patch_option):

    deg2rad = np.pi/180

    # patch_option = 1 = 145 patches (Robinson & Stone, 2004)
    # patch_option = 2 = 153 patches (Wallenberg et al., 2022)
    # patch_option = 3 = 306 patches -> test 
    # patch_option = 4 = 612 patches -> test

    skyvaultalt = np.atleast_2d([])
    skyvaultazi = np.atleast_2d([])
            
    # Creating skyvault of patches of constant radians (Tregeneza and Sharples, 1993)
    # Patch option 1, 145 patches, Original Robinson & Stone (2004) after Tregenza (1987)/Tregenza & Sharples (1993)
    if patch_option == 1:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90]) # Robinson & Stone (2004)
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0]) # Fredrik/Nils
        patches_in_band = np.array([30, 30, 24, 24, 18, 12, 6, 1]) # Robinson & Stone (2004)
    # Patch option 1, 153 patches, Wallenberg et al. (2022)
    elif patch_option == 2:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90]) # Robinson & Stone (2004)
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0]) # Fredrik/Nils
        patches_in_band = np.array([31, 30, 28, 24, 19, 13, 7, 1]) # Nils
    # Patch option 2, 306 patches, test
    elif patch_option == 3:
        annulino = np.array([0, 12, 24, 36, 48, 60, 72, 84, 90])
        skyvaultaltint = np.array([6, 18, 30, 42, 54, 66, 78, 90]) # Robinson & Stone (2004)
        azistart = np.array([0, 4, 2, 5, 8, 0, 10, 0]) # Fredrik/Nils
        patches_in_band = np.array([31*2, 30*2, 28*2, 24*2, 19*2, 13*2, 7*2, 1]) # Nils
    # Patch option 3, 612 patches, test
    elif patch_option == 4:
        annulino = np.array([0, 4.5, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 90]) # Nils
        skyvaultaltint = np.array([3, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 90]) # Nils
        patches_in_band = np.array([31*2, 31*2, 30*2, 30*2, 28*2, 28*2, 24*2, 24*2, 19*2, 19*2, 13*2, 13*2, 7*2, 7*2, 1]) # Nils
        azistart = np.array([0, 0, 4, 4, 2, 2, 5, 5, 8, 8, 0, 0, 10, 10, 0]) # Nils

    skyvaultaziint = np.array([360/patches for patches in patches_in_band])

    for j in range(0, skyvaultaltint.shape[0]):
        for k in range(0, patches_in_band[j]):
            skyvaultalt = np.append(skyvaultalt, skyvaultaltint[j])
            skyvaultazi = np.append(skyvaultazi, k*skyvaultaziint[j] + azistart[j])

    # skyvaultzen = (90 - skyvaultalt) * deg2rad
    # skyvaultalt = skyvaultalt * deg2rad
    # skyvaultazi = skyvaultazi * deg2rad

    return skyvaultalt, skyvaultazi, annulino, skyvaultaltint, patches_in_band, skyvaultaziint, azistart

In [4]:
def shadowingfunction_20(a, vegdem, vegdem2, azimuth, altitude, scale, amaxvalue, bush, feedback, forsvf):

    # plt.ion()
    # fig = plt.figure(figsize=(24, 7))
    # plt.axis('image')
    # ax1 = plt.subplot(2, 3, 1)
    # ax2 = plt.subplot(2, 3, 2)
    # ax3 = plt.subplot(2, 3, 3)
    # ax4 = plt.subplot(2, 3, 4)
    # ax5 = plt.subplot(2, 3, 5)
    # ax6 = plt.subplot(2, 3, 6)
    # ax1.title.set_text('fabovea')
    # ax2.title.set_text('gabovea')
    # ax3.title.set_text('vegsh at ' + str(altitude))
    # ax4.title.set_text('lastfabovea')
    # ax5.title.set_text('lastgabovea')
    # ax6.title.set_text('vegdem')

    # This function casts shadows on buildings and vegetation units.
    # New capability to deal with pergolas 20210827

    # conversion
    degrees = np.pi/180.
    azimuth = azimuth * degrees
    altitude = altitude * degrees
    
    # measure the size of grid
    sizex = a.shape[0]
    sizey = a.shape[1]
    
    # progressbar for svf plugin
    if forsvf == 0:
        barstep = np.max([sizex, sizey])
        total = 100. / barstep
        feedback.setProgress(0)
        # dlg.progressBar.setRange(0, barstep)
        # dlg.progressBar.setValue(0)

    # initialise parameters
    dx = 0.
    dy = 0.
    dz = 0.
    temp = np.zeros((sizex, sizey))
    tempvegdem = np.zeros((sizex, sizey))
    tempvegdem2 = np.zeros((sizex, sizey))
    templastfabovea = np.zeros((sizex, sizey))
    templastgabovea = np.zeros((sizex, sizey))
    bushplant = bush > 1.
    sh = np.zeros((sizex, sizey)) #shadows from buildings
    vbshvegsh = np.zeros((sizex, sizey)) #vegetation blocking buildings
    vegsh = np.add(np.zeros((sizex, sizey)), bushplant, dtype=float) #vegetation shadow
    f = a

    pibyfour = np.pi / 4.
    threetimespibyfour = 3. * pibyfour
    fivetimespibyfour = 5.* pibyfour
    seventimespibyfour = 7. * pibyfour
    sinazimuth = np.sin(azimuth)
    cosazimuth = np.cos(azimuth)
    tanazimuth = np.tan(azimuth)
    signsinazimuth = np.sign(sinazimuth)
    signcosazimuth = np.sign(cosazimuth)
    dssin = np.abs((1./sinazimuth))
    dscos = np.abs((1./cosazimuth))
    tanaltitudebyscale = np.tan(altitude) / scale
    # index = 1
    index = 0

    # new case with pergola (thin vertical layer of vegetation), August 2021
    dzprev = 0

    # main loop
    while (amaxvalue >= dz) and (np.abs(dx) < sizex) and (np.abs(dy) < sizey):
        if forsvf == 0:
            feedback.setProgress(int(index * total)) #dlg.progressBar.setValue(index)
        if ((pibyfour <= azimuth) and (azimuth < threetimespibyfour) or (fivetimespibyfour <= azimuth) and (azimuth < seventimespibyfour)):
            dy = signsinazimuth * index
            dx = -1. * signcosazimuth * np.abs(np.round(index / tanazimuth))
            ds = dssin
        else:
            dy = signsinazimuth * np.abs(np.round(index * tanazimuth))
            dx = -1. * signcosazimuth * index
            ds = dscos
        # note: dx and dy represent absolute values while ds is an incremental value
        dz = (ds * index) * tanaltitudebyscale
        tempvegdem[0:sizex, 0:sizey] = 0.
        tempvegdem2[0:sizex, 0:sizey] = 0.
        temp[0:sizex, 0:sizey] = 0.
        templastfabovea[0:sizex, 0:sizey] = 0.
        templastgabovea[0:sizex, 0:sizey] = 0.
        absdx = np.abs(dx)
        absdy = np.abs(dy)
        xc1 = int((dx+absdx)/2.)
        xc2 = int(sizex+(dx-absdx)/2.)
        yc1 = int((dy+absdy)/2.)
        yc2 = int(sizey+(dy-absdy)/2.)
        xp1 = int(-((dx-absdx)/2.))
        xp2 = int(sizex-(dx+absdx)/2.)
        yp1 = int(-((dy-absdy)/2.))
        yp2 = int(sizey-(dy+absdy)/2.)

        tempvegdem[xp1:xp2, yp1:yp2] = vegdem[xc1:xc2, yc1:yc2] - dz
        tempvegdem2[xp1:xp2, yp1:yp2] = vegdem2[xc1:xc2, yc1:yc2] - dz
        temp[xp1:xp2, yp1:yp2] = a[xc1:xc2, yc1:yc2]-dz

        f = np.fmax(f, temp) #Moving building shadow
        sh[(f > a)] = 1.
        sh[(f <= a)] = 0.
        fabovea = tempvegdem > a #vegdem above DEM
        gabovea = tempvegdem2 > a #vegdem2 above DEM
        
        #new pergola condition
        templastfabovea[xp1:xp2, yp1:yp2] = vegdem[xc1:xc2, yc1:yc2]-dzprev
        templastgabovea[xp1:xp2, yp1:yp2] = vegdem2[xc1:xc2, yc1:yc2]-dzprev
        lastfabovea = templastfabovea > a
        lastgabovea = templastgabovea > a
        dzprev = dz
        vegsh2 = np.add(np.add(np.add(fabovea, gabovea, dtype=float),lastfabovea, dtype=float),lastgabovea, dtype=float)
        vegsh2[vegsh2 == 4] = 0.
        # vegsh2[vegsh2 == 1] = 0. # This one is the ultimate question...
        vegsh2[vegsh2 > 0] = 1.

        vegsh = np.fmax(vegsh, vegsh2)
        vegsh[(vegsh * sh > 0.)] = 0.
        vbshvegsh = vegsh + vbshvegsh # removing shadows 'behind' buildings

        # im1 = ax1.imshow(fabovea)
        # im2 = ax2.imshow(gabovea)
        # im3 = ax3.imshow(vegsh)
        # im4 = ax4.imshow(lastfabovea)
        # im5 = ax5.imshow(lastgabovea)
        # im6 = ax6.imshow(vegshtest)
        # im1 = ax1.imshow(tempvegdem)
        # im2 = ax2.imshow(tempvegdem2)
        # im3 = ax3.imshow(vegsh)
        # im4 = ax4.imshow(templastfabovea)
        # im5 = ax5.imshow(templastgabovea)
        # im6 = ax6.imshow(vegshtest)
        # plt.show()
        # plt.pause(0.05)

        index += 1.

    sh = 1.-sh
    vbshvegsh[(vbshvegsh > 0.)] = 1.
    vbshvegsh = vbshvegsh-vegsh
    vegsh = 1.-vegsh
    vbshvegsh = 1.-vbshvegsh

    # plt.close()
    # plt.ion()
    # fig = plt.figure(figsize=(24, 7))
    # plt.axis('image')
    # ax1 = plt.subplot(1, 3, 1)
    # im1 = ax1.imshow(vegsh)
    # plt.colorbar(im1)

    # ax2 = plt.subplot(1, 3, 2)
    # im2 = ax2.imshow(vegdem2)
    # plt.colorbar(im2)
    # plt.title('TDSM')

    # ax3 = plt.subplot(1, 3, 3)
    # im3 = ax3.imshow(vegdem)
    # plt.colorbar(im3)
    # plt.tight_layout()
    # plt.title('CDSM')
    # plt.show()
    # plt.pause(0.05)

    shadowresult = {'sh': sh, 'vegsh': vegsh, 'vbshvegsh': vbshvegsh}

    return shadowresult

In [5]:
def annulus_weight(altitude, aziinterval):
    n = 90.
    steprad = (360./aziinterval) * (np.pi/180.)
    annulus = 91.-altitude
    w = (1./(2.*np.pi)) * np.sin(np.pi / (2.*n)) * np.sin((np.pi * (2. * annulus - 1.)) / (2. * n))
    weight = steprad * w

    return weight

def svf_angles_100121():
    azi1 = np.arange(1., 360., 360./16.)  #%22.5
    azi2 = np.arange(12., 360., 360./16.)  #%22.5
    azi3 = np.arange(5., 360., 360./32.)  #%11.25
    azi4 = np.arange(2., 360., 360./32.)  #%11.25
    azi5 = np.arange(4., 360., 360./40.)  #%9
    azi6 = np.arange(7., 360., 360./48.)  #%7.50
    azi7 = np.arange(6., 360., 360./48.)  #%7.50
    azi8 = np.arange(1., 360., 360./48.)  #%7.50
    azi9 = np.arange(4., 359., 360./52.)  #%6.9231
    azi10 = np.arange(5., 360., 360./52.)  #%6.9231
    azi11 = np.arange(1., 360., 360./48.)  #%7.50
    azi12 = np.arange(0., 359., 360./44.)  #%8.1818
    azi13 = np.arange(3., 360., 360./44.)  #%8.1818
    azi14 = np.arange(2., 360., 360./40.)  #%9
    azi15 = np.arange(7., 360., 360./32.)  #%10
    azi16 = np.arange(3., 360., 360./24.)  #%11.25
    azi17 = np.arange(10., 360., 360./16.)  #%15
    azi18 = np.arange(19., 360., 360./12.)  #%22.5
    azi19 = np.arange(17., 360., 360./8.)  #%45
    azi20 = 0.  #%360
    iazimuth = np.array(np.hstack((azi1, azi2, azi3, azi4, azi5, azi6, azi7, azi8, azi9, azi10, azi11, azi12, azi13,
                                    azi14, azi15, azi16, azi17, azi18, azi19, azi20)))
    aziinterval = np.array(np.hstack((16., 16., 32., 32., 40., 48., 48., 48., 52., 52., 48., 44., 44., 40., 32., 24.,
                                        16., 12., 8., 1.)))
    angleresult = {'iazimuth': iazimuth, 'aziinterval': aziinterval}

    return angleresult

In [6]:
def svfForProcessing153(dsm, vegdem, vegdem2, scale, usevegdem, feedback):
    rows = dsm.shape[0]
    cols = dsm.shape[1]
    svf = np.zeros([rows, cols])
    svfE = np.zeros([rows, cols])
    svfS = np.zeros([rows, cols])
    svfW = np.zeros([rows, cols])
    svfN = np.zeros([rows, cols])
    svfveg = np.zeros((rows, cols))
    svfEveg = np.zeros((rows, cols))
    svfSveg = np.zeros((rows, cols))
    svfWveg = np.zeros((rows, cols))
    svfNveg = np.zeros((rows, cols))
    svfaveg = np.zeros((rows, cols))
    svfEaveg = np.zeros((rows, cols))
    svfSaveg = np.zeros((rows, cols))
    svfWaveg = np.zeros((rows, cols))
    svfNaveg = np.zeros((rows, cols))

    # % amaxvalue
    vegmax = vegdem.max()
    amaxvalue = dsm.max()
    amaxvalue = np.maximum(amaxvalue, vegmax)

    # % Elevation vegdems if buildingDSM inclused ground heights
    vegdem = vegdem + dsm
    vegdem[vegdem == dsm] = 0
    vegdem2 = vegdem2 + dsm
    vegdem2[vegdem2 == dsm] = 0
    # % Bush separation
    bush = np.logical_not((vegdem2 * vegdem)) * vegdem

    index = int(0)

    # patch_option = 1 # 145 patches
    patch_option = 2 # 153 patches
    # patch_option = 3 # 306 patches
    # patch_option = 4 # 612 patches
    
    # Create patches based on patch_option
    skyvaultalt, skyvaultazi, annulino, skyvaultaltint, aziinterval, skyvaultaziint, azistart = create_patches(patch_option)

    skyvaultaziint = np.array([360/patches for patches in aziinterval])
    iazimuth = np.hstack(np.zeros((1, np.sum(aziinterval)))) # Nils

    shmat = np.zeros((rows, cols, np.sum(aziinterval)))
    vegshmat = np.zeros((rows, cols, np.sum(aziinterval)))
    vbshvegshmat = np.zeros((rows, cols, np.sum(aziinterval)))

    for j in range(0, skyvaultaltint.shape[0]):
        for k in range(0, int(360 / skyvaultaziint[j])):
            iazimuth[index] = k * skyvaultaziint[j] + azistart[j]
            if iazimuth[index] > 360.:
                iazimuth[index] = iazimuth[index] - 360.
            index = index + 1

            
    aziintervalaniso = np.ceil(aziinterval / 2.0)
    index = int(0)
    for i in range(0, skyvaultaltint.shape[0]):
        for j in np.arange(0, (aziinterval[int(i)])):
            if feedback.isCanceled():
                feedback.setProgressText("Calculation cancelled")
                break
            altitude = skyvaultaltint[int(i)]
            azimuth = iazimuth[int(index)]

            # Casting shadow
            if usevegdem == 1:
                shadowresult = shadowingfunction_20(dsm, vegdem, vegdem2, azimuth, altitude,
                                                           scale, amaxvalue, bush, feedback, 1)
                vegsh = shadowresult["vegsh"]
                vbshvegsh = shadowresult["vbshvegsh"]
                sh = shadowresult["sh"]
                vegshmat[:, :, index] = vegsh
                vbshvegshmat[:, :, index] = vbshvegsh
            else:
                sh = 0 #shadowingfunctionglobalradiation(dsm, azimuth, altitude, scale, feedback, 1)

            shmat[:, :, index] = sh

            # Calculate svfs
            for k in np.arange(annulino[int(i)]+1, (annulino[int(i+1.)])+1):
                weight = annulus_weight(k, aziinterval[i])*sh
                svf = svf + weight
                weight = annulus_weight(k, aziintervalaniso[i]) * sh
                if (azimuth >= 0) and (azimuth < 180):
                    svfE = svfE + weight
                if (azimuth >= 90) and (azimuth < 270):
                    svfS = svfS + weight
                if (azimuth >= 180) and (azimuth < 360):
                    svfW = svfW + weight
                if (azimuth >= 270) or (azimuth < 90):
                    svfN = svfN + weight

            if usevegdem == 1:
                for k in np.arange(annulino[int(i)] + 1, (annulino[int(i + 1.)]) + 1):
                    # % changed to include 90
                    weight = annulus_weight(k, aziinterval[i])
                    svfveg = svfveg + weight * vegsh
                    svfaveg = svfaveg + weight * vbshvegsh
                    weight = annulus_weight(k, aziintervalaniso[i])
                    if (azimuth >= 0) and (azimuth < 180):
                        svfEveg = svfEveg + weight * vegsh
                        svfEaveg = svfEaveg + weight * vbshvegsh
                    if (azimuth >= 90) and (azimuth < 270):
                        svfSveg = svfSveg + weight * vegsh
                        svfSaveg = svfSaveg + weight * vbshvegsh
                    if (azimuth >= 180) and (azimuth < 360):
                        svfWveg = svfWveg + weight * vegsh
                        svfWaveg = svfWaveg + weight * vbshvegsh
                    if (azimuth >= 270) or (azimuth < 90):
                        svfNveg = svfNveg + weight * vegsh
                        svfNaveg = svfNaveg + weight * vbshvegsh

            index += 1
            feedback.setProgress(int(index * (100. / np.sum(aziinterval))))

    svfS = svfS + 3.0459e-004
    svfW = svfW + 3.0459e-004
    # % Last azimuth is 90. Hence, manual add of last annuli for svfS and SVFW
    # %Forcing svf not be greater than 1 (some MATLAB crazyness)
    svf[(svf > 1.)] = 1.
    svfE[(svfE > 1.)] = 1.
    svfS[(svfS > 1.)] = 1.
    svfW[(svfW > 1.)] = 1.
    svfN[(svfN > 1.)] = 1.

    if usevegdem == 1:
        last = np.zeros((rows, cols))
        last[(vegdem2 == 0.)] = 3.0459e-004
        svfSveg = svfSveg + last
        svfWveg = svfWveg + last
        svfSaveg = svfSaveg + last
        svfWaveg = svfWaveg + last
        # %Forcing svf not be greater than 1 (some MATLAB crazyness)
        svfveg[(svfveg > 1.)] = 1.
        svfEveg[(svfEveg > 1.)] = 1.
        svfSveg[(svfSveg > 1.)] = 1.
        svfWveg[(svfWveg > 1.)] = 1.
        svfNveg[(svfNveg > 1.)] = 1.
        svfaveg[(svfaveg > 1.)] = 1.
        svfEaveg[(svfEaveg > 1.)] = 1.
        svfSaveg[(svfSaveg > 1.)] = 1.
        svfWaveg[(svfWaveg > 1.)] = 1.
        svfNaveg[(svfNaveg > 1.)] = 1.

    svfresult = {'svf': svf, 'svfE': svfE, 'svfS': svfS, 'svfW': svfW, 'svfN': svfN,
                    'svfveg': svfveg, 'svfEveg': svfEveg, 'svfSveg': svfSveg, 'svfWveg': svfWveg,
                    'svfNveg': svfNveg, 'svfaveg': svfaveg, 'svfEaveg': svfEaveg, 'svfSaveg': svfSaveg,
                    'svfWaveg': svfWaveg, 'svfNaveg': svfNaveg, 'shmat': shmat, 'vegshmat': vegshmat, 'vbshvegshmat': vbshvegshmat}
                    # ,
                    # 'vbshvegshmat': vbshvegshmat, 'wallshmat': wallshmat, 'wallsunmat': wallsunmat,
                    # 'wallshvemat': wallshvemat, 'facesunmat': facesunmat}
    return svfresult

In [7]:
def processAlgorithm(dsm_path, vegdsm_path, vegdsm2_path, output_dir, output_file, trans_veg, trunk_ratio, aniso):
    # InputParameters
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    # Load DSM
    gdal_dsm = gdal.Open(dsm_path)
    dsm = gdal_dsm.ReadAsArray().astype(float)
    nd = gdal_dsm.GetRasterBand(1).GetNoDataValue()
    dsm[dsm == nd] = 0.
    if dsm.min() < 0:
        dsm = dsm + np.abs(dsm.min())

    sizex, sizey = dsm.shape
    geotransform = gdal_dsm.GetGeoTransform()
    scale = 1 / geotransform[1]
    
    trans = trans_veg / 100.0

    if vegdsm_path:
        usevegdem = 1
        print('Vegetation scheme activated')

        # Load vegetation DSM
        vegdsm = gdal.Open(vegdsm_path).ReadAsArray().astype(float)
        vegsizex, vegsizey = vegdsm.shape

        if not (vegsizex == sizex and vegsizey == sizey):
            raise ValueError("Error in Vegetation Canopy DSM: All rasters must be of same extent and resolution")

        if vegdsm2_path:
            vegdsm2 = gdal.Open(vegdsm2_path).ReadAsArray().astype(float)
        else:
            vegdsm2 = vegdsm * (trunk_ratio / 100.0)

        vegsizex, vegsizey = vegdsm2.shape
        if not (vegsizex == sizex and vegsizey == sizey):
            raise ValueError("Error in Trunk Zone DSM: All rasters must be of same extent and resolution")
    else:
        vegdsm = np.zeros([dsm.shape[0], dsm.shape[1]])
        vegdsm2 = np.zeros([dsm.shape[0], dsm.shape[1]])
        usevegdem = 0

    # Perform SVF calculation
    if aniso == 1:
        print('Calculating SVF using 153 iterations')
        ret = svfForProcessing153(dsm, vegdsm, vegdsm2, scale, usevegdem)
    else:
        print("aniso=False")
        ret = 0 #svfForProcessing655(dsm, vegdsm, vegdsm2, scale, usevegdem) 

    if ret is not None:
        svfbu = ret["svf"]
        svfbuE = ret["svfE"]
        svfbuS = ret["svfS"]
        svfbuW = ret["svfW"]
        svfbuN = ret["svfN"]

        saveraster(gdal_dsm, os.path.join(output_dir, 'svf.tif'), svfbu)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfE.tif'), svfbuE)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfS.tif'), svfbuS)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfW.tif'), svfbuW)
        saveraster(gdal_dsm, os.path.join(output_dir, 'svfN.tif'), svfbuN)

        with zipfile.ZipFile(os.path.join(output_dir, 'svfs.zip'), 'w') as zipf:
            for file in ['svf.tif', 'svfE.tif', 'svfS.tif', 'svfW.tif', 'svfN.tif']:
                zipf.write(os.path.join(output_dir, file), file)
                os.remove(os.path.join(output_dir, file))

        if usevegdem == 0:
            svftotal = svfbu
        else:
            svfveg = ret["svfveg"]
            svfEveg = ret["svfEveg"]
            svfSveg = ret["svfSveg"]
            svfWveg = ret["svfWveg"]
            svfNveg = ret["svfNveg"]
            svfaveg = ret["svfaveg"]
            svfEaveg = ret["svfEaveg"]
            svfSaveg = ret["svfSaveg"]
            svfWaveg = ret["svfWaveg"]
            svfNaveg = ret["svfNaveg"]

            saveraster(gdal_dsm, os.path.join(output_dir, 'svfveg.tif'), svfveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfEveg.tif'), svfEveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfSveg.tif'), svfSveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfWveg.tif'), svfWveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfNveg.tif'), svfNveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfaveg.tif'), svfaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfEaveg.tif'), svfEaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfSaveg.tif'), svfSaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfWaveg.tif'), svfWaveg)
            saveraster(gdal_dsm, os.path.join(output_dir, 'svfNaveg.tif'), svfNaveg)

            with zipfile.ZipFile(os.path.join(output_dir, 'svfs.zip'), 'a') as zipf:
                for file in ['svfveg.tif', 'svfEveg.tif', 'svfSveg.tif', 'svfWveg.tif', 'svfNveg.tif', 'svfaveg.tif', 'svfEaveg.tif', 'svfSaveg.tif', 'svfWaveg.tif', 'svfNaveg.tif']:
                    zipf.write(os.path.join(output_dir, file), file)
                    os.remove(os.path.join(output_dir, file))

            svftotal = (svfbu - (1 - svfveg) * (1 - trans))

        saveraster(gdal_dsm, output_file, svftotal)

        # Save shadow images
        if aniso == 1:
            shmat = ret["shmat"]
            vegshmat = ret["vegshmat"]
            vbshvegshmat = ret["vbshvegshmat"]
            np.savez_compressed(os.path.join(output_dir, "shadowmats.npz"), shadowmat=shmat, vegshadowmat=vegshmat, vbshmat=vbshvegshmat)

    print("Sky View Factor: SVF grid(s) successfully generated")